In [7]:
import requests
import time
import os
import pandas as pd
import numpy as np
import random
import re

# !pip3 install -U spacy
import spacy
from spacy import displacy
from time import sleep
from os import path
from pandas import DataFrame
from bs4 import BeautifulSoup
from random import randint

In [2]:
def get_contents(soup, content_text):
  try:
    parents_blacklist=['[document]','html','head',
                       'style','script','body',
                       'section','tr',
                       'td','label','ul','header',
                       'aside']
    content=''
    text=soup.find_all(text=True)

    
    for t in text:
        if t.parent.name not in parents_blacklist and len(t) > 5:
            content=content+t+' '
    content_text.append(content)
  except Exception:
    content_text.append('')
    pass

In [52]:
def preprocessor_final(text):
    if isinstance((text), (str)):
        text = re.sub('<[^>]*>', ' ', text)
#         text = re.sub('[\W_^\$]+', ' ', text.lower())
        return text
    if isinstance((text), (list)):
        return_list = []
        for i in range(len(text)):
            temp_text = re.sub('<[^>]*>', ' ', text[i])
#             text = re.sub('[\W_^\$]+', ' ', text.lower())
            return_list.append(temp_text)
        return(return_list)
    else:
        pass

In [19]:
def display_entities(model_path, content):
    nlp = spacy.load(model_path)
    doc = nlp(content)
    displacy.render(doc, style="ent", jupyter = True)
    

In [61]:
page_2015 ="https://web.archive.org/web/20150217144133/https://www.apartments.com/chicago-il/"
page_2022 = 'https://web.archive.org/web/20220819053652/https://www.apartments.com/chicago-il/'

page=requests.get(page_2015)
soup=BeautifulSoup(page.text,'html.parser')
content_text = []
get_contents(soup, content_text)
content = "".join(content_text)

In [56]:
clean_content = preprocessor_final(content)
clean_content = ' '.join(clean_content.split())

In [57]:
display_entities("en_core_web_lg", clean_content)

In [58]:
display_entities("models/model-best-all-fields", clean_content)

In [59]:
display_entities("models/model-best", clean_content)